In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Flatten
from tensorflow.keras.preprocessing import sequence
#from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SpatialDropout1D
#from mlxtend.plotting import plot_confusion_matrix
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [8]:
MAX_WORDS = 100
MAX_LEN = 100
BATCH_SIZE=64
EPOCHS=100

In [9]:
df=pd.read_csv("/Users/abhinavshinow/Documents/GitHub/Mal_URL/Data/XSS_dataset.csv")

In [10]:
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)

In [11]:
df.head()

,Sentence,Label
0,"<li><a href=""/wiki/File:Socrates.png"" class=""i...",0
1,"<tt onmouseover=""alert(1)"">test</tt>",1
2,"\t </span> <span class=""reference-text"">Steeri...",0
3,"\t </span> <span class=""reference-text""><cite ...",0
4,"\t </span>. <a href=""/wiki/Digital_object_iden...",0


In [20]:
x=df['Sentence']
y=df['Label']

#Tokenising the url's
tok = Tokenizer(num_words=MAX_WORDS)
tok.fit_on_texts(x)
x = tok.texts_to_sequences(x.values)
x = sequence.pad_sequences(x, maxlen=MAX_LEN)

#One-Hot Encoding the target labels
#y = np.array(to_categorical(y))

x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,shuffle=True)

In [21]:
x_train.shape

(10948, 100)

In [22]:
model = Sequential()
# model.add(Embedding(MAX_WORDS, 300, input_length=MAX_LEN))
# model.add(SpatialDropout1D(0.2))
# model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.1,return_sequences=True,activation='relu'))
# model.add(Dense(32,activation='relu'))
# model.add(Dense(4,activation='softmax'))
# model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
# print(model.summary())
model = Sequential()
model.add(Embedding(MAX_WORDS, 300, input_length=MAX_LEN))
model.add(SpatialDropout1D(0.1))
model.add(LSTM(32, dropout=0.4, recurrent_dropout=0.1,return_sequences=True, activation='relu'))
model.add(Flatten())
model.add(Dense(1,activation="sigmoid"))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 300)          30000     
_________________________________________________________________
spatial_dropout1d_2 (Spatial (None, 100, 300)          0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100, 32)           42624     
_________________________________________________________________
flatten_2 (Flatten)          (None, 3200)              0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3201      
Total params: 75,825
Trainable params: 75,825
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
es=EarlyStopping(
        monitor='val_loss', 
        mode='min', 
        patience=3,
        restore_best_weights=True, 
        verbose=1,
    )
checkpoint_cb=ModelCheckpoint(
        "embedding_ulr.h5",
        save_best_only=True,
        monitor='val_loss',
        mode='min',
    )
reduce_lr=ReduceLROnPlateau(
        monitor='val_loss',
        patience=6,
        mode='min',
        verbose=1,
    )
callbacks=[es,checkpoint_cb,reduce_lr]

In [9]:
# steps_per_epoch = int( np.ceil(x_train.shape[0] / BATCH_SIZE) )

In [24]:
history = model.fit(x_train, y_train, batch_size=BATCH_SIZE, epochs = EPOCHS,steps_per_epoch=128,validation_data=(x_test,y_test),callbacks=callbacks)

Epoch 1/100


2021-11-30 08:29:05.201942: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:112] Plugin optimizer for device_type GPU is enabled.


 71/128 [===============>..............] - ETA: 21:23 - loss: 0.0000e+00 - accuracy: 0.4562